In [1]:
import onnx
import os
import wandb
import onnxruntime as ort
import numpy as np
from utils_tbm import *
from test_tbm import test as evals

In [2]:
model = onnx.load(os.getcwd()+"/model.onnx")
(onnx.checker.check_model(model))

In [ ]:
print(model)

In [4]:

train_data, _ = get_data_from_csv('../../preprocessed.csv', train=True)
test_data, _ = get_data_from_csv('../../preprocessed.csv', train=False)
test_loader = make_loader(test_data, batch_size=32)

In [5]:
remote_hosts = get_test_remote_hosts('../../preprocessed.csv')

In [7]:
print(len(remote_hosts))

4015


In [45]:
features_data = test_loader.dataset.tensors[0]

# Convert to float, add an extra dimension, and convert to numpy array
data_numpy = features_data.float().unsqueeze(0).numpy()
reshaped_data = data_numpy.reshape((4015, 20, 25))
print(data_numpy.shape)


(1, 4015, 20, 25)


In [46]:
session = ort.InferenceSession("model.onnx")

# Reshape data if necessary to match the input shape expected by the model
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# Run inference
predictions = session.run([output_name], {input_name: reshaped_data})

In [59]:
mse_loss = np.mean(np.square(reshaped_data - predictions))
threshold = 0.00001 
if mse_loss > threshold:
    print("Data is considered anomalous.")
else:
    print("Data is within normal range.")



Data is considered anomalous.


In [61]:
predictions = session.run([output_name], {input_name: reshaped_data})

# Initialize a dictionary to hold the indices of anomalous sequences mapped to their remote_host
anomalous_sequences = {}

# Loop through each sequence in the reshaped data
for i in range(predictions[0].shape[0]):
    # Get the prediction for the current sequence
    pred_seq = predictions[0][i]
    
    # Compare the prediction to the original sequence
    orig_seq = reshaped_data[i]
    
    # Calculate the absolute difference between the prediction and the original sequence
    diff = np.abs(pred_seq - orig_seq)
    
    # Sum the differences across all elements of the sequence
    sum_diff = np.sum(diff)
    
    # If the sum of differences exceeds a threshold, flag the sequence as anomalous
    if sum_diff > 0.001:
        # Store the index of the anomalous sequence in the dictionary, keyed by remote_host
        anomalous_sequences[remote_hosts_ordered[i]] = i

# Print the indices of the anomalous sequences for each remote_host
for host, indices in anomalous_sequences.items():
    print(f"{host} has anomalous sequences at indices: {indices}")

NameError: name 'remote_hosts_ordered' is not defined